# Curso de Python - Parte 2

## 8. Ficheros y entrada/salida, apertura de ficheros e introducción de contextos.

Para abrir un fichero usamos la función `open()`, que tiene como parámetros el nombre y ruta del fichero y el modo de apertura, una serie de caracteres que definen como se va a usar el fichero abierto.

```python
>>> f = open('workfile', 'w')
```

- `r` para solo lectura
- `w` para solo escritura, siendo borrado cualquier fichero que ya existiera
- `a` para añadir contenido al final del fichero
- `r+` para lectura y escritura

Por defecto, si se omite el modo, se asume que es `r`. 

Además, también por defecto, un fichero se abre en modo texto, lo que significa que lee el contenido como cadenas de texto, aplicando transformaciones que dependen de la plataforma, como es la conversión de los salos de líneas, entre los formatos de Unix o de Windows, aplicando esta transformación tanto a la lectura como a la escritura.


Para abrir en modo **binario** añadimos `b` al final de la cadena de texto que indica el modo.

La función `open` devuelve un descriptor que apunta al fichero, no el contenido. Una vez se deja de usar se tiene que llamar al método `f.close()` del descriptor para cerrar el fichero.


Una buena práctica, para evitar la necesidad de cerrar el descriptor, es usar la palabra reservada `with`. Esto crea lo que se define en Python como un **contexto**.

```python
>>> with open('workfile') as f:
...     read_data = f.read()
>>> f.closed
True
```

En este ejemplo, la variable `f` solo existe en el bloque que se ejecuta dentro del `with`, y se llama a `f.close()` una vez que la ejecución sale del bloque.

### Lectura de contenido

#### Método `read(size)`

Lee la cantidad de información del fichero indicada en el parámetro size. Si se omite, lee todo el contenido del fichero.


#### Método `readline()`

Cada vez que se llama, lee la siguiente linea del fichero, es decir, hasta el siguiente salto de línea.

Para iterar sobre las líneas de un fichero, es más eficiente hacerlo directamente sobre el descriptor del fichero.

```python
>>> for line in f:
...     print(line, end='')
...
This is the first line of the file.
Second line of the file
```

Puedes leer todas las lineas de un fichero y pasarlas a una lista con `list(f)` o `f.readlines()`.

### Escritura de contenido

#### Método `write(string)`

El método `write(string)` escribe el contenido de string en el fichero, devolviendo el número de caracteres escritos.

### Moverse por el fichero

El método `f.tell()` devuelve un entero con la posición en la que nos encontramos en el fichero, representado por el número de bytes desde el comienzo, cuando se abre en modo binario.

Con el método `f.seek(offset, from_what)` nos podemos desplazar por el fichero. La posición a la que nos movemos se calcula añadiendo lo indicado en el offset a la posición refernciada en from_what, que es 0 para el comienzo del fichero, 1 para la posición actual del descriptor y 2 para el final del fichero.


## 9. Módulos y paquetes, organización del código en el proyecto con paquetes y módulos y como importar funciones.

Hasta ahora, cuando salimos del interprete y volvemos a entrar, todas las definiciones y declaraciones que hemos realizado se pierden. Para escribir cualquier programa que vaya más allá de lo que el uso puntual del interprete requiera, debemos escribirlo en un fichero de texto con la extensión `.py`.

El interprete de Python tratara cada uno de estos ficheros como un módulo, `module`. Cada definición de nombres, variables, funciones, etc. que contenga un módulo puede ser importado a otro módulo y ser usado en este.

Por ejemplo, si tenemos el siguiente fichero, `fibo.py`:

```python
# Módulo números de Fibonacci


def fib(n):
    a, b = 0, 1
    while b < n:
        print(b, end=' ')
        a, b = b, a+b
    print()

        
def fib2(n):
    result = []
    a, b = 0, 1
    while b < n:
        result.append(b)
        a, b = b, a+b
    return result
```

- Podemos importar estas funciones en el intérprete o en cualquier otro módulo.
- Cada módulo define una variable global `__name__`, que contiene el nombre del módulo.

```python
>>> import fibo
>>> fibo.fib(1000)
1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987
>>> fibo.fib2(100)
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
>>> fibo.__name__
'fibo'
```

Un módulo puede tener instrucciones además de definiciones. Estas instrucciones suelen tener como objetivo inicializar el módulo, y **sólo se ejecutarán la primera vez que se hace un `import` del módulo**. También se ejecutan cuando el fichero de módulo se ejecuta como un script.

Todo módulo tiene su propia tabla de símbolos, por lo que cualquiera puede usar variables globales sin preocuparse de que estas colisionen entre módulos.


Se pueden importar nombres desde un módulo directamente a la tabla de nombres de otro módulo.

```python
>>> from fibo import fib, fib2
>>> fib(500)
1 1 2 3 5 8 13 21 34 55 89 144 233 377
```

Y también se pueden importar todos los nombres que define un módulo.

```python
>>> from fibo import *
>>> fib(500)
1 1 2 3 5 8 13 21 34 55 89 144 233 377
```

Esto importa todos los nombres excepto los que empiezan por `_`.

### Ejecutar módulos como scripts

Puedes ejecutar cualquier módulo usando el comando de `python`.

```
$ python fibo.py
```

Si el código del módulo se ejecuta de esta forma, entonces la variable `__name__` del módulo toma el valor de `__main__`, lo que significa que podemos diferenciar que parte del modulo se ejecutará cuando se importa y cuando se ejecuta como script.

```python
if __name__ == "__main__":
    # Sólo ejecuta esto cuando es un script
    pass 
```

### Paquetes

Los paquetes o *packages* de Python son una forma de estructurar los módulos de Python. 

Se considera un package cualquier carpeta que tenga un fichero llamado `__init__.py`. Este fichero puede estar vacío, pero también puede incluir código para inicializar el paquete. Un paquete puede a su vez contener, o no, uno o varios módulos.

```
sound/                          
      __init__.py               
      formats/                  
              __init__.py
              wavread.py
              wavwrite.py
              aiffread.py
              aiffwrite.py
              auread.py
              auwrite.py
              ...
      effects/                  
              __init__.py
              echo.py
              surround.py
              reverse.py
              ...
      filters/                  
              __init__.py
              equalizer.py
              vocoder.py
              karaoke.py
              ...
```

Si se intenta hacer un `from sound.effects import *` cabría esperar que esto importara todos los nombres definidos en todos los módulos de dentro del paquete y sus sub-paquetes, pero sería muy costoso, por que se sigue la convención de que el autor de un paquete declare la variable `__all__` como una lista de nombres de los módulos que se importarán cuando se use el * para importar todo.

## 10. Programación orientada a objetos

Las clases son un medio que nos permite juntar datos con funcionalidad. Cuando creas una nueva clase estás creando un nuevo tipo de objeto, permitiendo que puedan crear nuevas instancias de este. Cada clase puede tener:

- Atributos, que mantienen su estado, y
- métodos, definidos en las clases, que permiten modificar el estado.

### Nombres y objetos

Un objeto puede tener diferentes nombres en diferentes lugares del programa. Esto es debido a que, como comentábamos al principio, Python internamente, cuando asignamos un valora una variable, estamos en realidad asociando un nombre a un objeto en concreto.


Esto es similar al funcionamiento de los punteros en otros lenguajes de programación. Además, permite que pasar objetos a una función sea barato, ya que sólo se está pasando un "puntero" al objeto, de forma que si se modifica dentro de la función, el cambio se ve reflejado fuera.

Esto se puede aplicar a todos los objetos, excepto a los tipos básicos inmutables, como los números, cadenas de texto y tuplas.

In [1]:
class Dummy:
    def __init__(self, value):
        self.value = value

def modify_dummy(obj, value):
    obj.value = value

def modify_var(var, value):
    var = value

def modify_list(l, value):
    l.append(value)

dummy = Dummy(1)
print(dummy.value)
modify_dummy(dummy, 2)
print(dummy.value)

var = 3
print(var)
modify_var(var, 4)
print(var)


some_list = [1, 2, 3]
print(some_list)
modify_list(some_list, 4)
print(some_list)

1
2
3
3
[1, 2, 3]
[1, 2, 3, 4]


### Namespaces y scopes

Antes de entrar en detalle en las clases, conviene entender como funcionan las reglas de alcance en Python, ya que las clases hacen uso de estos conceptos para funcionar.

Un **namespace** es un mapa que asocia nombres a objetos, implementado internamente usando diccionarios de Python. No existe ninguna relación entre los nombres definidos en diferentes **namespaces**.

Un **scope** es una región de un programa de Python desde donde un **namespace** se puede acceder directamente. En todo momento de la ejecución de un programa, hay siempre al menos tres **scopes** anidados:

1. El scope local, el más interno, que contiene las definiciones de los nombres locales.
2. El scope de cualquier función dentro de la que estemos, empezando por la más interna a la más externa.
3. El scope que contiene los nombres a nivel de módulo.
4. El scope que contiene los nombres reservados del lenguaje.

Hay formas de hacer referencia a scopes diferentes, usando `global` y `nonlocal`.

In [2]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("Después de la asignación local:", spam)
    do_nonlocal()
    print("Después de la asignación nonocal:", spam)
    do_global()
    print("Después de la asignación global:", spam)

scope_test()
print("En el scope global:", spam)

Después de la asignación local: test spam
Después de la asignación nonocal: nonlocal spam
Después de la asignación global: nonlocal spam
En el scope global: global spam


## 11. Clases y objetos, diferencia entre la instancia, el objeto clase, y la declaración de clases.


La forma más sencilla de definir una clase sería:

In [ ]:
class ClassName:
    pass

Cuando se entra en la definición de una clase, se crea un nuevo namespace, y se accede a él desde el scope local, por lo que toda asignación o definición que se haga, quedará será accesible sólo desde ese scope local.

Cuando se define una clase decimos que se crea un objeto clase, que se queda asociado en el namespace al nombre que le hemos dado a la clase, en nuestro caso `ClassName`.

Un objeto clase soporta dos tipos de operaciones:

- referencia a atributos, mediante el operador `.`
- instanciación, usando los paréntesis, de la misma forma que se usan en una función

In [3]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'

print(MyClass.i)
print(MyClass.f)

12345
<function MyClass.f at 0x10a306510>


En este ejemplo, `MyClass.i` sería una referencia válida a un atributo de clase, y `MyClass.f` sería una referencia válida a un método.

Para crear una nueva instancia de la clase, simplemente usamos la notación de la función:

```python
x = MyClass()
```

Esto crea una nueva instancia de la clase, y guarda este objeto en la variable local x.

In [4]:
x = MyClass()  # nueva instancia de MyClass
print(x.i)
print(x.f())

12345
hello world


## 12. Atributos y métodos, la declaración, el self, los métodos estáticos, de clase, etc.

Cuando instanciamos una clase podemos hacer que la nueva instancia tenga un estado inicial concreto.

Para ello debemos definir un método `__init__()`.

In [ ]:
class MyClass:
    def __init__(self):
        self.data = []

- Siempre que se instancia una clase, se llama al método `__init__`. 
- El primer argumento que recibe este método, así como en todos los métodos de la clase es `self`. 
- El argumento `self` hace referencia siempre a la instancia que está ejecutando el método en cada instante.

In [5]:
class MyClass:

    def __init__(self):
        self.data = []

    def size(self):
        return len(self.data)


x = MyClass()  # al llamar a __init__, self hace referencia a la intancia recien creada que se asociará al nombre x
x.data.append("a")
y = MyClass()  # y tendrá una nueva lista vacía en data
print(x.size())  # muestra 1
print(y.size())  # muestra 0

1
0


Este método `__init__` también acepta argumentos, que pueden ser pasados cuando se crea una nueva instancia de la clase, como si se trataran de los argumentos de una función.

A un objeto instancia se le pueden agregar nuevos atributos o métodos de forma dinámica, simplemente referenciando un nuevo nombre con el operador `.`. No hace falta que estén declarados o inicializados en el `__init__`.

In [10]:
x = MyClass()
x.value = 42
print(x.value)  # muestra 42

42


Además de estos métodos de instancia, que reciben el argumento `self` para hacer referencia a la instancia actual, se pueden declarar métodos estáticos y métodos de clase.

Los métodos estáticos se declaran con la notación `@staticmethod` sobre la definición, y no reciben el argumento `self`.

Los métodos de clase se declarar con la notación `@classmethod` sobre la definición, y en vez de `self`, reciben `cls` que hace referencia al objeto clase actual

In [11]:
class MyClass:

    def __init__(self):
        self.data = []

    @staticmethod
    def some_static_stuff():
        print("Soy estático!")

    @classmethod
    def some_class_stuff(cls):
        cls.class_attribute = "Soy un nuevo atributo de clase!"

MyClass.some_static_stuff()
MyClass.some_class_stuff()
print(MyClass.class_attribute)

Soy estático!
Soy un nuevo atributo de clase!


## 13. Herencia y herencia múltiple, sistema de herencia de Python.

Python soporta la herencia entre clases, y para declarar una clase que hereda de otra lo podemos hacer de la siguiente forma:

```python
class BaseClaseName:
    pass


class DerivedClassName(BaseClaseName):
    pass
```

Cuando una clase derivada o hija se construye, esta recuerda el namespace de la clase padre, por lo que tiene acceso a los mismos nombres que esta clase define.

Si se definen atributos o métodos con los mismo nombres, estas nuevas definiciones sobreescriben su asociación en el namespace, por lo que podemos decir que podemos cambiar la implementación de los métodos de la clase base.

Python soporta también la herencia múltiple. Una herencia múltiple puede ser definidia de la siguiente forma:

```python
class Base1:
    pass


class Base2:
    pass


class Base3:
    pass


class DerivedClassName(Base1, Base2, Base3):
    pass
```

Cuando una clase que tiene herencia múltiple intenta buscar un nombre en su namespace, realiza la búsqueda en profundidad y de izquierda a derecha, es decir, en el ejemplo, primero busca el nombre en Base1, luego en la ascendencia de Base1, luego en Base2, luego en la ascendencia de Base2 y por último en Base3 y su ascendencia.

En un método cualquiera, podemos acceder a la implementación del padre usando la función `super()`.

## 14. Encapsulación, métodos y atributos "privados", con _ o __ al principio.

En el sentido tradicional, todos los métodos y atributos de un objeto en Python se consideran públicos, es decir, pueden ser accedidos desde cualquier punto, mientras se tenga una referencia al objeto.

Sin embargo, se usa una convección para definir métodos y atributos privados, que serían todos aquellos que empiezan con al menos un `_` en el nombre. Todos estos atributos o métodos deberán ser tratados como parte no pública del API de la clase, de tal forma que la implementación de este método se podría cambiar sin que un usuario de esta clase lo notara.

Todo identificador que empiece por `__`, dos `_`, será traducido internamente por `__classname__`, de tal forma que si tenemos un método `__spam`, sería traducido internamente por `__classname__spam`. Esto impide que un método que empiece por `__` pueda ser sobreescrito por una clase hija. También es útil si queremos cambiar los argumentos que recibe un método en una clase hija, sin romper la clase padre.

In [59]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)
    
    __update = update

class MappingSubclass(Mapping):

    def update(self, keys, values):
        # proporciona un nuevo método update, con parámetros distintos
        # pero no rompe la implementación de __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

    __update = update
    

Mapping.__dict__

mappingproxy({'_Mapping__update': <function __main__.Mapping.update>,
              '__dict__': <attribute '__dict__' of 'Mapping' objects>,
              '__doc__': None,
              '__init__': <function __main__.Mapping.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Mapping' objects>,
              'update': <function __main__.Mapping.update>})

In [47]:
MappingSubclass.__dict__

mappingproxy({'_MappingSubclass__update': <function __main__.MappingSubclass.update>,
              '__doc__': None,
              '__module__': '__main__',
              'update': <function __main__.MappingSubclass.update>})

In [58]:
MappingSubclass([1, 2, 3])

Esto es una convención que se usa, y que se el propio interprete respeta en cierta medida, pero cabe resaltar que el hecho de que se declare un método como privado no impide que sea accedido desde fuera de la clase.

## 15. Métodos mágicos

El método `__init__` visto anteriormente se le considera también un método mágico. Los métodos mágicos de las clases permiten definir el comportamiento de esta en diferentes circunstancias, como la inicialización, la comparación con otros objetos o como obtener una representación.

### Construcción e inicialización

**`__new__(cls, [...)`**

Este es el primer método que se llama al crear una instancia de una clase, antes de que la instancia en si exista. Recibe la clase y los mismos argumentos que se le pasan a `__init__`.

**`__init__(self, [...)`**

Es el inicializador de la instancia. Se llama cuando ya existe la instancia, y tiene como argumentos los que se le pasan al crear la instancia. Este junto a `__new__` formarían el constructor como lo entendemos en otros lenguajes de programación.

`__del__(self)`

Este método se llama cuando el colector de basura destruye el objeto. No se llama cuando se hace uso de `del`. No se recomienda su uso ya que no se puede garantizar que sea llamado cuando el objeto existe y se cierra el interprete.


### Comparaciones

`__eq__(self, other)`

Permite realizar una comparación igualdad entra el objeto y `other`.


`__ne__(self, other)`

Permite realizar una comparación no igualdad entra el objeto y `other`.

`__lt__(self, other)`

Permite realizar una comparación menor que entra el objeto y `other`.

`__gt__(self, other)`

Permite realizar una comparación mayor que entra el objeto y `other`.

`__le__(self, other)`


Permite realizar una comparación menor o igual que entra el objeto y `other`.

`__ge__(self, other)`

Permite realizar una comparación mayor o igual que entra el objeto y `other`.


### Operadores unarios

`__pos__(self)`

Realiza y devuelve el resultado de la operación `+obj`.

`__neg__(self)`

Realiza y devuelve el resultado de la operación `-obj`.

`__abs__(self)`

Realiza y devuelve el resultado de la operación `abs(obj)`.

`__invert__(self)`

Realiza y devuelve el resultado de la operación `~obj`.

`__round__(self, n)`

Realiza y devuelve el resultado de la operación `round(obj)`.

`__floor__(self)`

Realiza y devuelve el resultado de la operación `math.floor(obj)`.

`__ceil__(self)`

Realiza y devuelve el resultado de la operación `math.ceil(obj)`.

`__trunc__(self)`

Realiza y devuelve el resultado de la operación `math.trunc(obj)`.

### Operadores aritméticos normales

`__add__(self, other)`

Realiza y deveulve el resultado de hacer `obj + other`.

`__sub__(self, other)`

Realiza y deveulve el resultado de hacer `obj - other`.

`__mul__(self, other)`

Realiza y deveulve el resultado de hacer `obj * other`.

`__floordiv__(self, other)`

Realiza y deveulve el resultado de hacer `obj // other`.

`__div__(self, other)`

Realiza y deveulve el resultado de hacer `obj / other`.

`__mod__(self, other)`

Realiza y deveulve el resultado de hacer `obj % other`.

`__divmod__(self, other)`

Realiza y deveulve el resultado de hacer `divmod(obj, other)`.

`__pow__`

Realiza y deveulve el resultado de hacer `obj ** other`.

`__lshift__(self, other)`

Realiza y deveulve el resultado de hacer `obj << other`.

`__rshift__(self, other)`

Realiza y deveulve el resultado de hacer `obj >> other`.

`__and__(self, other)`

Realiza y deveulve el resultado de hacer `obj & other`.

`__or__(self, other)`

Realiza y deveulve el resultado de hacer `obj | other`.

`__xor__(self, other)`

Realiza y deveulve el resultado de hacer `obj ^ other`.

### Operadores aritméticos reflejados

`__radd__(self, other)`

Realiza y deveulve el resultado de hacer `other + obj`.

`__rsub__(self, other)`

Realiza y deveulve el resultado de hacer `other - obj`.

`__rmul__(self, other)`

Realiza y deveulve el resultado de hacer `other * obj`.

`__rfloordiv__(self, other)`

Realiza y deveulve el resultado de hacer `other // obj`.

`__rdiv__(self, other)`

Realiza y deveulve el resultado de hacer `other / obj`.

`__rmod__(self, other)`

Realiza y deveulve el resultado de hacer `other % obj`.

`__rdivmod__(self, other)`

Realiza y deveulve el resultado de hacer `divmod(other,  obj)`.

`__rpow__`

Realiza y deveulve el resultado de hacer `other ** obj`.

`__rlshift__(self, other)`

Realiza y deveulve el resultado de hacer `other << obj`.

`__rrshift__(self, other)`

Realiza y deveulve el resultado de hacer `other >> obj`.

`__rand__(self, other)`

Realiza y deveulve el resultado de hacer `other & obj`.

`__ror__(self, other)`

Realiza y deveulve el resultado de hacer `other | obj`.

`__rxor__(self, other)`

Realiza y deveulve el resultado de hacer `other ^ obj`.

### Operadores con asignación

`__iadd__(self, other)`

Realiza y deveulve el resultado de hacer `obj += other`.

`__isub__(self, other)`

Realiza y deveulve el resultado de hacer `obj -= other`.

`__imul__(self, other)`

Realiza y deveulve el resultado de hacer `obj *= other`.

`__ifloordiv__(self, other)`

Realiza y deveulve el resultado de hacer `obj //= other`.

`__idiv__(self, other)`

Realiza y deveulve el resultado de hacer `obj /= other`.

`__imod__(self, other)`

Realiza y deveulve el resultado de hacer `obj %= other`.

`__ipow__`

Realiza y deveulve el resultado de hacer `obj **= other`.

`__ilshift__(self, other)`

Realiza y deveulve el resultado de hacer `obj <<= other`.

`__irshift__(self, other)`

Realiza y deveulve el resultado de hacer `obj >>= other`.

`__iand__(self, other)`

Realiza y deveulve el resultado de hacer `obj &= other`.

`__ior__(self, other)`

Realiza y deveulve el resultado de hacer `obj |= other`.

`__ixor__(self, other)`

Realiza y deveulve el resultado de hacer `obj ^= other`.

### Representación de Clases

**`__str__(self)`**

Se llama para obtener una representación en texto de la clase cuando se llama a `str(obj)`.

`__bytes__(self)`

Se llama para obtener una representación como secuencia de bytes de la clase.

`__repr__(self)`

Se llama para obtener una representación en texto de la clase cuando se llama a `repr(obj)`.

`__format__(self, formatstr)`

Se llama cuando se hace llama a `obj.format()`.

`__hash__(self)`

Se llama cuando se hace llama a `hash(obj)`.

`__bool__(self)`

Se llama cuando se hace llama a `bool(obj)`.

`__dir__(self)`

Se llama cuando se hace llama a `dir(obj)`.

`__sizeof__(self)`

Se llama cuando se hace llama a `sizeof(obj)`.


### Acceso a atributos

**`__getattr__(self, name)`**

Define el comportamiento que realzia la clase cuando se accede a un atributo que no existe.

**`__setattr__(self, name, value)`**

Permite definir el comportamiento para la asignación de un valor en un atributo, tanto si existe como si no.

`__delattr__(self, name)`

Permite definir el comportamiento para cuando se intenta borrar un atributo, usando `del`.

**`__getattribute__(self, name)`**

Permite definir el comportamiento para el acceso a un atributo, tanto si existe como si no.

### Secuencias

`__len__(self)`

Devuelve el resultado de llamar a la función `len(ojb)`.

`__getitem__(self, key)`

Devuelve el resultado de llamar a  `obj[key]`.

`__setitem__(self, key, value)`

Devuelve el resultado de llamar a  `obj[key] = value`.

`__delitem__(self, key)`

Devuelve el resultado de llamar a  `del obj[key]`.


**`__iter__(self)`**

Devuelve el resultado de llamar a  `iter(obj)`.

`__reversed__(self)`

Devuelve el resultado de llamar a  `reversed(obj)`.

`__contains__(self, item)`

Devuelve el resultado de llamar a  `item in obj` o `item not in obj`.

`__missing__(self, key)`

Se utiliza en las subclases de `dic`, y se llama cuando se hace acceso a un elemento con clave `key` que no extiste en el diccionario.

### Objetos llamables

**`__call__(self, [args...])`**

Se ejecuta cuando se llama a una instancia como si fuera una función.

### Contextos

`__enter__(self)`

Se llama cuando se entra en un contexto con la instrucción `with`.

`__exit__(self, exception_type, exception_value, traceback)`

Se llama cuando se sale de un contexto.


## Ejercicios

### Palabras más frecuentes

Junto a este *notebook* se incluye un fichero con el contenido del libro "La Isla del Tesoro" en inglés. Lee el fichero y extrae las 30 palabras más frecuentes de todo el libro.

#### Ayuda

Python incluye un módulo llamado `re` para el uso de expresiones regulares. Puedes investigar como funciona y usarlo para resolver este ejercicio.

### Títulos de los capítulos

Teniendo en cuenta que en el libro anterior, los cápitulos siempre están de la siguiente forma:

```
{numero del capítulo}\n
\n
{titulo del capítulo}```

Muestra una lista con todos los títulos y números de los capítulos de "La Isla del Tesoro".

### Persistir datos en un JSON

Python proporciona una librería estándar para la gestión de ficheros JSON, llamado `json`.

In [ ]:
import json

json_content = '{"user": {"messages": []}}'
loads = json.loads(json_content)
print(loads, type(loads))

json_dict = {"user": {"messages": []}}
dumps = json.dumps(json_dict)
print(dumps, type(dumps))


El objetivo de este ejercicio es crear una clase que cargue un fichero JSON y permita el acceso a este usando directamente como si fuera un diccionario.

Además, la clase tendrá un método para guardar los cambios relaizados en el contenido cargado de nuevo en el fichero.

### Simulador de notificaciones


El objetivo de este ejercicio es desarrollar un **simulador de entrega de notificaciones a usuarios**. El simulador permitirá definir un entorno con varios usuarios a los que se le pueden entregar notificaciones y cada una de estas notificaciones tendrá una serie de estadísticas relacionadadas con los mensajes enviados, los recibidos y los abiertos.


- Registrar **usuarios**.
    - Cada usario tendrá un nombre, que será usado para su representación.
    - Un usuario será registrado en un listado único y se le asignará un código.
    - Cada usuario tendrá una bandeja de entrada con los mensajes que ha recibido.
    - Cada usuario podrá comprobar su bandeja de entrada y marcar un mensaje como leído.

- Crear y enviar **mensajes**:
    - Los mensajes tendrán una cadena de texto como cuerpo, que será usada para su representación.
    - Los mensajes tendrán una lista de usuarios a los que se les ha enviado el mensaje.
    - Los mensajes tendrán una lista de usuarios a los que han recibido el mensaje.
    - Los mensajes tendrán una lista de usuarios a los que han abierto el mensaje.
    - Dado un mensaje, se podrán obtener las estadísticas de apertura y recepción del mensaje.

- El **sitema de entrega de mensajes** estará separado de los usuarios y los mensajes, y deberá:
    - Mantener un registro de los usuarios registrados, asignando sus códigos.
    - Publicar un mensaje a todos los usuarios registrados, asignando en ese proceso un código único a cada mensaje, y añdiendo acada mensaje en la bandeja de entrada del usuario.
    - Simular un ratio de pérdida de mensajes, es decir, que exista una probabilidad de que un mensaje no sea recibido por un usuario.

Además de las definiciones de las clases necesarias, habrá que implementar una simulación de envío de mensajes a 1000 usuarios y mostrar las estadísticas.

Hay que tener en cuenta que cada uno de los usuarios a los que le llege el mensaje tendrá que poder obtener una lista de los mensajes que se le han enviado, sabiendo si lo ha leído o no.

#### Ayuda

Para crear identificadores únicos, podemos usar el módulo `uuid` de Python:

```python
>>> import uuid
>>> str(uuid.uuid1())
'9d6a768a-3659-11e8-966c-60f81db53974'
```

### Soporte para envíos individualizados en simulador de notificaciones

En el escenario planteado anteriormente sólo se da soporte para simular envíos en *broadcast*. Añade una nueva clase de mensaje y modifica el servidor para permitir el envío de mensajes individualizados.

### Módulos para el simulador de mensajes

El objetivo de este ejercico es tomar el código que se ha desarrollado en el ejercicio anterior y escribirlo en diferentes módulos.

Uno de los módulos deberá de permitir ser ejecutado como script para lanzar una simulación.